# Common filters
 
Uses filters that are commonly used in Paraview such as gradient calculations and also uses the raw data and the Numpy package to compute the $\lambda_2$ vortex identification criterion which is given by where the second largest eigenvalue of the matrix 
$$S_{ik}S_{kj} + \Omega_{ik}\Omega_{kj}$$
is negative where 

$$S_{ij} = \frac{1}{2}\left(\frac{\partial u_{i}}{\partial x_j} + \frac{\partial u_{j}}{\partial x_i}\right)$$
$$\Omega_{ij} = \frac{1}{2}\left(\frac{\partial u_{i}}{\partial x_j} - \frac{\partial u_{j}}{\partial x_i}\right)$$



In [1]:
import pyvista as pv
import numpy as np
fn = 'postprocessing/RESULTS_FLUID_DOMAIN.case'

reader = pv.get_reader(fn)
reader.set_active_time_value(300)

data = reader.read()['Fluid domain']

print(data)

UnstructuredGrid (0x75275657b880)
  N Cells:    3140920
  N Points:   3172091
  X Bounds:   -1.000e+00, 1.000e+00
  Y Bounds:   -1.000e+00, 1.000e+00
  Z Bounds:   0.000e+00, 2.000e+01
  N Arrays:   10


In [2]:
# Q criterion

Q = data.compute_derivative('Velocity',qcriterion='Q', gradient=False)
print(Q.cell_data)

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3140920,)           SCALARS
    Velocity                float32    (3140920, 3)         VECTORS
    Pressure                float32    (3140920,)
    TurbVisc                float32    (3140920,)
    CourantNb               float32    (3140920,)
    FourierNb               float32    (3140920,)
    total_pressure          float32    (3140920,)
    U_mean                  float32    (3140920, 3)
    UU_mean                 float32    (3140920, 6)
    P_mean                  float32    (3140920,)
    Q                       float32    (3140920,)


In [3]:
Q_contour = Q.ctp().contour([0.5],
                      scalars='Q')

In [4]:
p = pv.Plotter(window_size=(500,500))

p.add_mesh(Q_contour,
           scalars='Velocity',
           cmap='bwr')

p.add_axes()
p.view_xy(negative=True)
p.camera.azimuth = 30
p.camera.elevation = 30
p.show()

Widget(value='<iframe src="http://localhost:42373/index.html?ui=P_0x7527566906d0_0&reconnect=auto" class="pyvi…

In [5]:
# lambda2

dudx_data = data.compute_derivative('Velocity',gradient='dudx')
print(dudx_data.cell_data)

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3140920,)           SCALARS
    Velocity                float32    (3140920, 3)         VECTORS
    Pressure                float32    (3140920,)
    TurbVisc                float32    (3140920,)
    CourantNb               float32    (3140920,)
    FourierNb               float32    (3140920,)
    total_pressure          float32    (3140920,)
    U_mean                  float32    (3140920, 3)
    UU_mean                 float32    (3140920, 6)
    P_mean                  float32    (3140920,)
    dudx                    float32    (3140920, 9)


In [6]:
dudx = dudx_data.cell_data['dudx'].reshape((dudx_data.n_cells ,3,3))

dudxT = np.transpose(dudx,axes=(0,2,1))
S = 0.5*(dudx + dudxT)
Omega = 0.5*(dudx - dudxT)

In [7]:
S2O2 = np.matmul(S,S) + np.matmul(Omega,Omega)

eig, _ = np.linalg.eigh(S2O2)

lambda2 = eig[:,1]

dudx_data.cell_data['lambda_2'] = lambda2

In [8]:

lambda2_contour = dudx_data.ctp().contour([-0.4],
                                    scalars='lambda_2')

p = pv.Plotter(window_size=(500,500))

p.add_mesh(lambda2_contour,
           scalars='Velocity',
           cmap='bwr')

p.add_axes()
p.view_xy(negative=True)
p.camera.azimuth = 30
p.camera.elevation = 30
p.show()


Widget(value='<iframe src="http://localhost:42373/index.html?ui=P_0x752743c0c550_1&reconnect=auto" class="pyvi…